### 第一个分类器

In [1]:
import fasttext

In [50]:
# 模型训练
model = fasttext.train_supervised(input="data/cooking.train")

In [21]:
# 保存模型
model.save_model("model_cooking.bin")

In [22]:
# 测试模型
model.predict("Which baking dish is best to bake a banana bread ?")

(('__label__baking',), array([0.08202819]))

In [23]:
model.predict("Why not put knives in the dishwasher ?")

(('__label__food-safety',), array([0.05188132]))

该模型所预测的标签是食品安全，这是不相关的。不知何故，这个模型似乎在一些简单的例子上失败了。

In [24]:
# 验证数据
model.test("cooking.valid")

(3000, 0.146, 0.06313968574311662)

上面的第一个参数是样本数，第二个为精度（单位1），第三个为召回率（单位1）

In [51]:
print("这次训练后的单词数为：%d，标签数为：%d" % (len(model.words), len(model.labels)))

这次训练后的单词数为：14543，标签数为：735


In [27]:
# 设置精度
model.test("cooking.valid", k=5)

(3000, 0.0674, 0.14574023353034454)

### 高级：准确率和召回率的计算

准确率是由fasttext预测的标签中正确标签的数量。召回率是所有真正的标签中成功预测的标签数量。

"Why not put knives in the dishwasher?"，这句话中有三个标签：equipment（设备）、cleaning（清洁）和knives（刀具）。该模型预测的前五个标签可以通过以下方式获得：

In [28]:
model.predict("Why not put knives in the dishwasher ?",k=5)

(('__label__food-safety',
  '__label__baking',
  '__label__bread',
  '__label__equipment',
  '__label__substitutions'),
 array([0.05188132, 0.05018667, 0.0325066 , 0.027121  , 0.02521329]))

上面预测的五个标签中（food-safety 食品安全、baking 烘焙、equipment 设备、substitutions 替代品和bread 面包），只有一个标签是正确的准确率为0.027。

在三个真正的标签中，只有一个在这个模型中被预测到了，召回率为0.33。

### 让模型变得更好

由上面的样例可以看出，使用默认参数运行fasttext所获得的模型在分类新问题时非常糟糕。我们可以通过更改参数来改进性能。

#### 预处理数据

通过查看数据，我们可以发现一些单词中包含大写字母或标点符号。改进模型性能的第一步是应用一些简单的预处理。可以使用shell命令如sed和tr

```shell
cat data/cooking.stackexchange.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > cooking.preprocessed.txt

head -n 12404 cooking.preprocessed.txt > cooking.train

tail -n 3000 cooking.preprocessed.txt > cooking.valid
```


In [2]:
# 使用预处理的数据，训练新的模型
import fasttext
model = fasttext.train_supervised(input="cooking.train")

In [49]:
print("这次训练后的单词数为：%d，标签数为：%d" % (len(model.words), len(model.labels)))

这次训练后的单词数为：8952，标签数为：735


由数据对比得知，预处理之后的模型中的单词为8952，预处理之前的单词数为14543，准确率上升了2.4%

In [55]:
model = fasttext.train_supervised(input="cooking.train", lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')
model.test("cooking.valid")

(3000, 0.589, 0.25472106097736774)

In [56]:
model = fasttext.train_supervised(input="cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
model.test("cooking.valid")

(3000, 0.6126666666666667, 0.2649560328672337)

In [60]:
model = fasttext.train_supervised(input="cooking.train", lr=0.3, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
model.test("cooking.valid")

(3000, 0.606, 0.262072942194032)

In [1]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [3]:
print_results(*model.test('cooking.valid'))

N	3000
P@1	0.176
R@1	0.076
